# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [2]:
import sqlite3    # Python interface for SQLite databases

In [3]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [4]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
  4: REGION(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
sale:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
seller:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [5]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [6]:
%%sql SELECT *
FROM Master1
WHERE NV IS NOT NULL
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
1,Mercurey,11.5,1980,300,1,Bohn,Rene,Alsace
1,Mercurey,11.5,1980,1,73,Bourguignon,Emile,Jura
2,Julienas,11.3,1974,100,5,Joudeat,Lucien,Bourgogne
3,Savigny les Beaunes,12.1,1978,400,1,Bohn,Rene,Alsace
4,Mercurey,10.9,1980,35,10,Fournier,Jean Claude,Bourgogne
5,Pommard,11.7,1976,46,30,Desbois,Maurice,Bourgogne
6,Mercurey,11.2,1981,300,42,Juillot,Maurice,Bourgogne
7,Grands Echezeaux,11.7,1968,60,98,Berard,None,Rhone
8,Cotes de Beaune Villages,12.3,1975,12,90,Roure,Raymond,Rhone
9,Chapelle Chambertin,11.9,1973,None,None,None,None,None


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [7]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

> ANSWER:
- The Master1 Table contains information about producers, wine and how many bottles where harvest by each producer.
To fit the 2NF, 3NF and BCNF. We should make 3 tables for each.
- The table Producers will contain (NP, NOM, PRENOM, REGION)
- The table Wine will contain (NV, CRU, DEGRE, MILL)
- The table Harvest will contain (ID_HARV, NP, NV, QTE)
As we can see, NP and NV are the primary keys of Producers and Wine, and are related to the table Harvest. Where they are foreign keys

Concerning functional dependencies:
In the table Producers we have: NP->{PRENOM, NOM, REGION}<br/>
In the table Wine we have: NV -> {CRU, MILL, DEGRE}<br/>
In the table Harvest we have: {NV, NP} -> QTE<br/>
A good practice could be to set a primary key for the table harvest.

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

> ANSWER:
- The Master2 Table contains information about customer, wine and how many bottles where buyed by each customer.
To fit the 2NF, 3NF and BCNF. We should make 3 tables for each.
- The table Customer will contain (NB, NOM, PRENOM, TYPE, REGION)
- The table Wine will contain (NV, CRU, DEGRE, MILL) (same as the Master1
- The table Buy will contain (ID_BUY, NB, NV, QTE, LIEU, DATES)
As we can see, NB and NV are the primary keys of Customer and Wine, and are related to the table Buy. Where they are foreign keys

Concerning functional dependencies:
In the table Customer we have: NB->{PRENOM, NOM, REGION, TYPE}<br/>
In the table Wine we have: NV -> {CRU, MILL, DEGRE}<br/>
In the table Buy we have: {NB, NV} -> {QTE, LIEU, DATES}<br/>
A good practice could be to set a primary key for the table Buy.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [8]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [9]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
  4: REGION(TEXT)
dummy:
  0: DEGRE(NUM)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
sale:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
seller:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


In [10]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [11]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [12]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [13]:
%%sql DROP TABLE IF EXISTS wine;
      DROP TABLE IF EXISTS producer;
      DROP TABLE IF EXISTS production;

-- Create wine table
CREATE TABLE wine AS
SELECT DISTINCT NV,CRU,DEGRE,MILL
FROM MASTER1
WHERE NV IS NOT NULL;


-- Create producer table
CREATE TABLE producer AS
SELECT DISTINCT NP,NOM,PRENOM,REGION
FROM MASTER1
WHERE NP IS NOT NULL;


-- Create production table
CREATE TABLE production AS
SELECT DISTINCT NV,NP,QTE
FROM MASTER1
WHERE NV IS NOT NULL AND NP IS NOT NULL;

SELECT *
FROM production
LIMIT(10);

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.


NV,NP,QTE
1,1,300
1,73,1
2,5,100
3,1,400
4,10,35
5,30,46
6,42,300
7,98,60
8,90,12
10,98,100


#### Exercise 1.4

Create the new tables from Master2:

In [24]:
%%sql DROP TABLE IF EXISTS client;
      DROP TABLE IF EXISTS sale;

-- Create client table
CREATE TABLE client AS
SELECT DISTINCT NB,NOM,PRENOM,TYPE
FROM MASTER2
WHERE NB IS NOT NULL;


-- Create sale table
CREATE TABLE sale AS
SELECT DISTINCT NB,NV,DATES,LIEU,REGION,QTE
FROM MASTER2
WHERE NB IS NOT NULL AND NV IS NOT NULL;

SELECT *
FROM sale
LIMIT(10);

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.


NB,NV,DATES,LIEU,REGION,QTE
2,1,1977-11-02,BORDEAUX,NOUVELLE-AQUITAINE,33
44,1,2015-10-16,PARIS,ÎLE-DE-FRANCE,1
45,1,1983-12-31,RENNES,BRETAGNE,1
48,2,1983-12-25,LYON,AUVERGNE-RHÔNE-ALPES,2
7,3,1978-11-01,NICE,PROVENCE-ALPES-CÔTE D'AZUR,6
49,3,1987-12-24,MERCUREY,BOURGOGNE-FRANCHE-COMTÉ,5
8,4,1982-11-05,TOULOUSE,OCCITANIE,12
44,4,2015-10-16,PARIS,ÎLE-DE-FRANCE,6
50,4,1981-06-29,SENS,BOURGOGNE-FRANCHE-COMTÉ,4
2,5,1984-11-04,MERCUREY,BOURGOGNE-FRANCHE-COMTÉ,50


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [28]:
printSchema(conn)

MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
sale:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: REGION(TEXT)
  5: QTE(NUM)
seller:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


In [25]:
%%sql

SELECT DISTINCT TYPE
FROM client
LIMIT(10);

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [26]:
%%sql
SELECT * 
FROM wine,production,producer
LIMIT(5)

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,NV_1,NP,QTE,NP_1,NOM,PRENOM,REGION
1,Mercurey,11.5,1980,1,1,300,3,Six,Paul,Alsace
1,Mercurey,11.5,1980,1,1,300,6,Marmagne,Bernard,Bourgogne
1,Mercurey,11.5,1980,1,1,300,8,Lioger d'Harduy,Gabriel,Bourgogne
1,Mercurey,11.5,1980,1,1,300,16,Barbin,Bernard,Bourgogne
1,Mercurey,11.5,1980,1,1,300,17,Faiveley,Guy,Bourgogne


In [27]:
%%sql

SELECT CRU,REGION
FROM wine 
JOIN production ON wine.NV=production.NV 
JOIN producer ON producer.NP= production.NP
WHERE CRU ='Pommard' OR CRU ='Brouilly';

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [29]:
%%sql

SELECT producer.REGION 
FROM producer
INNER JOIN production ON producer.NP = production.NP
INNER JOIN wine ON production.NV = wine.NV
WHERE wine.cru="Pommard" OR wine.cru="Brouilly"
GROUP BY producer.REGION HAVING COUNT (*) > 1

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bought by CRU and Millésime

In [34]:
%%sql
SELECT SUM(sale.QTE) AS QTE, CRU, MILL 
FROM sale
INNER JOIN wine ON wine.NV = sale.NV
GROUP BY wine.MILL, wine.CRU 
ORDER BY QTE DESC
LIMIT 3;

 * sqlite:///wine.db
Done.


QTE,CRU,MILL
520,Beaujolais Villages,1979
130,Beaujolais Villages,1978
120,Beaujolais Villages,1976


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [36]:
%%sql
SELECT wine.NV, wine.cru, COUNT(producer.NP) AS number_of_producers
FROM producer
INNER JOIN production ON producer.NP = production.NP
INNER JOIN wine ON production.NV = wine.NV
GROUP BY production.NV HAVING COUNT (producer.NP) > 3
LIMIT 3;

 * sqlite:///wine.db
Done.


NV,CRU,number_of_producers
45,Chiroubles,5
78,Etoile,5
89,Cotes de Provence,4


#### Exercise 2.6

Which producers have not produced any wine?

In [38]:
%%sql
SELECT NP, NBP, NOM 
FROM (SELECT DISTINCT producer.NP, producer.NOM, SUM(production.QTE) AS NBP FROM producer
LEFT JOIN production on producer.NP = production.NP
GROUP BY(producer.NP))
WHERE NBP IS NULL
LIMIT 3;

 * sqlite:///wine.db
Done.


NP,NBP,NOM
3,None,Six
6,None,Marmagne
8,None,Lioger d'Harduy


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [47]:
%%sql
SELECT client.NB,client.NOM, client.PRENOM 
FROM client
INNER JOIN sale on sale.NB=client.NB
INNER JOIN wine on wine.NV=sale.NV
WHERE wine.MILL=1980
GROUP BY client.NB
LIMIT 6;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [54]:
%%sql
SELECT client.NB,client.NOM, client.PRENOM
FROM client
WHERE client.NB NOT IN (SELECT client.NB
                        FROM client
                        INNER JOIN sale on sale.NB=client.NB
                        INNER JOIN wine on wine.NV=sale.NV
                        WHERE wine.MILL=1980
                        GROUP BY client.NB)
ORDER BY client.NB
LIMIT 5;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [65]:
%%sql
SELECT client.NB,client.NOM, client.PRENOM 
FROM client
INNER JOIN sale ON sale.NB=client.NB
INNER JOIN wine w ON w.NV=sale.NV
GROUP BY NOM, PRENOM
EXCEPT
SELECT client.NB,client.NOM, client.PRENOM
FROM client
INNER JOIN sale ON sale.NB=client.NB
INNER JOIN wine w ON w.NV=sale.NV
WHERE w.MILL!=1980
GROUP BY NOM, PRENOM
LIMIT 3;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [68]:
%%sql
SELECT *
FROM wine
WHERE MILL=1980
ORDER BY NV asc
LIMIT 3;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [70]:
%%sql
SELECT * 
FROM wine
INNER JOIN sale ON wine.NV = sale.NV
WHERE MILL=1980 AND sale.NB=2
GROUP BY wine.NV;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,NB,NV_1,DATES,LIEU,REGION,QTE
1,Mercurey,11.5,1980,2,1,1977-11-02,BORDEAUX,NOUVELLE-AQUITAINE,33


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [80]:
%%sql
SELECT client.NB, client.PRENOM, client.NOM, COUNT(wine.NV) AS Wine_2018
FROM client
INNER JOIN sale ON sale.NB = client.NB
INNER JOIN wine ON sale.NV=wine.NV
WHERE wine.MILL=1980
GROUP BY client.NB HAVING Wine_2018=(SELECT COUNT(wine.NV)
                                    FROM wine
                                    WHERE wine.MILL=1980)

 * sqlite:///wine.db
Done.


NB,PRENOM,NOM,Wine_2018
44,Andre,Gide,18
